# Introdução ao PySpark

*** Atenção: *** Utilize Java JDK 1.8.0_202 e Apache Spark 2.4.0

** Caso receba mensagem de erro "name 'sc' is not defined", interrompa o pyspark e apague o diretório metastore_db no mesmo diretório onde está este Jupyter notebook **

In [1]:
import sys
print(sys.version)

3.7.1 (default, Dec 14 2018, 19:28:38) 
[GCC 7.3.0]


In [2]:
print(sc)

<SparkContext master=local[*] appName=PySparkShell>


In [3]:
print(sc.version)

2.4.0


In [4]:
# Testando o spark e criando um RDD
lst = [25,98,81,37,776,3320]
testData = sc.parallelize(lst)

In [5]:
?sc.parallelize

In [6]:
type(testData)

pyspark.rdd.RDD

In [7]:
testData.count()

6

In [8]:
# Visualizar os dados
testData.collect()

[25, 98, 81, 37, 776, 3320]

RDD's são coleções distribuídas de itens. RDD's podem ser criadas a partir do Hadoop (arquivos no HDFS), através da transformação de outras RDD's, a partir de bancos de dados (relacionais e não-relacionais) ou a partir de arquivos locais.

In [9]:
# Criando uma RDD a partir de um arquivo csv
sentimentoRDD = sc.textFile("data/sentimentos.csv")

In [10]:
sentimentoRDD.count()

100

In [11]:
# Ação - Contando o número de registros
sentimentoRDD.count()

100

In [12]:
sentimentoRDD.collect()

['positivo,Esse livro é incrível.',
 'positivo,Um dos melhores livros que eu já li.',
 'positivo,um dos melhores livros que eu já li',
 'positivo,Acho que ele tem um conteúdo que vai além do que está em sua descrição.',
 'positivo,O Sol é para todos é profundo e emocionante',
 'positivo,Me encantei por este livro',
 'positivo,Este livro, apesar de antigo e com uma história que trata de uma época mais antiga ainda, possui um conteúdo atual e de extrema importância.',
 'positivo,The Da Vinci Code is actually a good movie...',
 'positivo,I thought the Da Vinci Code was a pretty good book.',
 'positivo,Um livro que realmente deveria ser lido por todas as pessoas.',
 'positivo,The Da Vinci Code is an * amazing * book',
 'positivo,then I turn on the light and the radio and enjoy my Da Vinci Code.',
 'positivo,Harper Lee aborda temas muito polêmicos, como o racismo e o Bullying, pelos olhos inocentes de uma criança de 7 anos.',
 'positivo,i love da vinci code....',
 'positivo,i loved da vinci

In [13]:
# Listando os primeiros registros
sentimentoRDD.take(5)

['positivo,Esse livro é incrível.',
 'positivo,Um dos melhores livros que eu já li.',
 'positivo,um dos melhores livros que eu já li',
 'positivo,Acho que ele tem um conteúdo que vai além do que está em sua descrição.',
 'positivo,O Sol é para todos é profundo e emocionante']

In [14]:
# Transformando os dados - transformação para letras maiúsculas
# A RDD é imutável, por isso temos que criar uma nova RDD (coleção de elementos)

tranfRDD = sentimentoRDD.map(lambda x: x.upper())
tranfRDD.take(5)

['POSITIVO,ESSE LIVRO É INCRÍVEL.',
 'POSITIVO,UM DOS MELHORES LIVROS QUE EU JÁ LI.',
 'POSITIVO,UM DOS MELHORES LIVROS QUE EU JÁ LI',
 'POSITIVO,ACHO QUE ELE TEM UM CONTEÚDO QUE VAI ALÉM DO QUE ESTÁ EM SUA DESCRIÇÃO.',
 'POSITIVO,O SOL É PARA TODOS É PROFUNDO E EMOCIONANTE']

In [15]:
sentimentoRDD.take(5)

['positivo,Esse livro é incrível.',
 'positivo,Um dos melhores livros que eu já li.',
 'positivo,um dos melhores livros que eu já li',
 'positivo,Acho que ele tem um conteúdo que vai além do que está em sua descrição.',
 'positivo,O Sol é para todos é profundo e emocionante']

In [16]:
arquivo = sc.textFile("data/sentimentos.csv")

In [17]:
type(arquivo)

pyspark.rdd.RDD

In [18]:
arquivo.count()

100

In [19]:
arquivo.first()

'positivo,Esse livro é incrível.'

In [20]:
# Operação de filtro - Sempre necessário criar um novo RDD
linhasComSol = arquivo.filter(lambda line: 'Sol' in line)

In [21]:
type(linhasComSol)

pyspark.rdd.PipelinedRDD

In [22]:
linhasComSol.count()

3

Primeiro a função map() determina o comprimento de cada linha do arquivo, criando uma RDD. A função reduce() é chamada para encontrar a linha com maior número de caracteres. O argumento para as funções map() e reduce() são funções anônimas criadas com lambda (da linguagem Python).

In [23]:
# Este exemplo é apenas em tempo de execução
arquivo.map(lambda line: len(line.split())).reduce(lambda a, b: a if(a > b) else b)

27

Está linha pode ser reescrita da sequinte forma

In [24]:
def max(a,b):
    if a > b:
        return a
    else:
        return b

arquivo.map(lambda line: len(line.split())).reduce(max)

27

## Operação de MapReduce

As operações de MapReduce foram popularizadas pelo Hadoop e podem ser feitas com Spark até 100x mais rápido.

In [25]:
contaPalavras = arquivo.flatMap(lambda line: line.split()).map(lambda palavra: (palavra, 1)).reduceByKey(lambda a, b: a + b)

In [27]:
type(contaPalavras)

pyspark.rdd.PipelinedRDD

In [28]:
contaPalavras.collect()

[('livro', 5),
 ('que', 13),
 ('li.', 4),
 ('positivo,um', 3),
 ('li', 1),
 ('positivo,Acho', 1),
 ('tem', 1),
 ('um', 3),
 ('vai', 1),
 ('do', 2),
 ('em', 1),
 ('descrição.', 1),
 ('positivo,O', 2),
 ('para', 5),
 ('todos', 4),
 ('positivo,Me', 1),
 ('este', 1),
 ('livro,', 1),
 ('antigo', 1),
 ('uma', 4),
 ('história', 1),
 ('antiga', 1),
 ('positivo,The', 6),
 ('Da', 38),
 ('Vinci', 45),
 ('Code', 24),
 ('is', 17),
 ('good', 3),
 ('movie...', 1),
 ('thought', 2),
 ('was', 4),
 ('pretty', 1),
 ('book.', 4),
 ('realmente', 1),
 ('deveria', 1),
 ('todas', 1),
 ('as', 1),
 ('pessoas.', 1),
 ('an', 6),
 ('*', 2),
 ('book', 2),
 ('turn', 1),
 ('positivo,Harper', 1),
 ('aborda', 1),
 ('muito', 3),
 ('polêmicos,', 1),
 ('como', 1),
 ('Bullying,', 1),
 ('olhos', 1),
 ('inocentes', 1),
 ('positivo,i', 4),
 ('love', 6),
 ('da', 13),
 ('code....', 1),
 ('loved', 5),
 ('code..', 2),
 ('VINCI', 4),
 ('BEAUTIFUL', 1),
 ('positivo,THE', 1),
 ('slash', 1),
 ('race.', 1),
 ('positivo,Hey', 1),
 ('The